## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [96]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')
print(df.info())
#print(df.head(10))
unique_children = df['children'].unique()
#print(unique_children)
unique_days_employed = df['days_employed'].unique()
#print(unique_days_employed)
#print(df['days_employed'].isna().sum())
unique_dob_years = df['dob_years'].unique()
#print(unique_dob_years)
unique_education = df['education'].unique()
#print(unique_education)
unique_education_id = df['education_id'].unique()
unique_family_status = df['family_status'].unique()
#print(unique_family_status)
unique_family_status_id = df['family_status_id'].unique()
unique_gender = df['gender'].unique()
unique_income_type = df['income_type'].unique()
#print(unique_income_type)
unique_debt = df['debt'].unique()
#print(unique_debt)
unique_total_income = df['total_income'].unique()
unique_purpose = df['purpose'].unique()
#print(unique_total_income)
#print(df['total_income'].isna().sum())
#print(unique_purpose)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Столбец children:

    1)Имеет отрицательные значения - вряд ли у родителя может быть -1 детей
Столбец days_employed:

    1)Имеет отрицательные значения. Вряд ли стаж сотрудника можеть быть -1 день.
    2)Имеет значения NaN - пропуски. Их следует заменить на 0
    3)Имеет неправильный тип данных. Вряд ли у человека можеть быть 1.5 дней стажа
Cтолбец dob_years:

    1)Имеет значение "0" - Может плохо влиять на статистику среднего возраста- Например 75 и 0 => mean = 32.5. Хотя 0 говорит об пропуски заполнения. 
Столбец education:

    1)Имеет одинаковые по значению строки, но с разными регистрами.
Столбец family_status:

    1)Категория "Не женат / не замужем" - единсвенная категория где первая буква имеет верхний регистр. Не критично, но все остальные данные имеют нижний ригистр
Столбец total_income:

    1)Имеет пропуски - Nan
Столбец purpose:

    1)Много дубликатов. Пример - 'покупка жилья' и 'операции с жильем'. Здесь стоит зделать список уникальных значений - применить лемматизацию. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [99]:
def abs(x):
    if x<0:
        return x*(-1)
    return x

df['children'] = df['children'].apply(abs)
#print(df['children'].unique())

df['days_employed'] = df['days_employed'].fillna(value=0)
df['days_employed'] = df['days_employed'].apply(abs)
#print(df['days_employed'].isna().sum())
#print(df.[df['days_employed']<0].count())

#print(df['dob_years'].value_counts())
print('Самый часто встречающий возраст: {}'.format(df['dob_years'].mode()[0]))
df.loc[df['dob_years'] == 0,'dob_years'] = df['dob_years'].mode()[0]

df.loc[df['family_status'] == 'Не женат / не замужем','family_status'] = 'не женат / не замужем'
#print(df['family_status'].unique())

df['total_income'] = df['total_income'].fillna(value=0)
#print(df['days_employed'].isna().sum())

Самый часто встречающий возраст: 35
       children  days_employed  dob_years education  education_id  \
0             1    8437.673028         42    высшее             0   
1             1    4024.803754         36   среднее             1   
2             0    5623.422610         33   Среднее             1   
3             3    4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1    4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1    2113.346888         38   среднее             1   
21523         3    3112.481705         38   среднее             1   
21524         2    1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     

### Вывод

Обработка пропусков была следующая:

1)Избавились от пропусков в столбце days_employed и total_income. Пропуски заполнены с помощью метода fillna().Подобно тому, как метод pandas dropna() управляет и удаляет значения Null из фрейма данных, fillna() управляет и позволяет пользователю заменять значения NaN на свои собственные значения. Пропуски могли появиться из за отсутсвия заполненого поля пользователем.

2)Так же была проведена замена отрицательных чисел на положительные в столбце children и days_employed.Вероятность получения отрицательных значений скорее всего является особенной выгрузкой данных.

3)С помощью метода mode() был определен самый часто встречаемый возраст и после пропуски возрастов равным 0. Были заменены на возраст 35.

4)У столюца family_status было произведено переименование уникального значения 'Не женат / не замужем' на 'не женат / не замужем' для избавления единственного верхнего регистра среди всех значений.

### Замена типа данных

In [100]:

df['days_employed'] = df['days_employed'].astype('int')
#print(df['days_employed'].unique())
#print(df.info())

df['total_income'] = df['total_income'].astype('int')
#print(df['total_income'].unique())
#print(df.info())

### Вывод

Была произведена замена типа столбца days_employed из float на тип integer. Так как у пользователя не можеть быть 1.5 дня стажа.
Для замены в нужный тип значения переводят методом astype(). Нам нужно было получить определенный тип Integer.Для этого и использовался метод astype(), у которого входной параметр - это тип значения на который нужно поменять.

### Обработка дубликатов

In [101]:

print(df.duplicated().sum())
df['education'] = df['education'].str.lower()
print(df.duplicated().sum())
df = df.drop_duplicates()
print(df.duplicated().sum())


71
0


### Вывод

Была произведена обработка дубликатов. Были удалены дубликаты столбца education. 
Поиск дубликатов был произведен с помощью метода value_counts(). Метод анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке.
Удаленые они были с помощью метода str.lower(). Метод берет строку и переводит в нижний регистр. Поскольку разница была толька в регистрах не пришлость применять дополнительный столб для удаления дубликатов(как было описано в тренажере). Затем после того как привели регистр к одному надо проверить дубликаты строчек. Т.е. что могло быть. 2 записи об одном заёмщике, но с разными регистрами. После того как их привели в один метод dublicate их увидет. Дубли надо удалять, это часть предообработки. 
Дубликат данных произошел из-за обычного ввода пользователем. Который может ввести слова с большой буквы, так и полностью написать все слово верхним регистром.

### Лемматизация

In [102]:
from pymystem3 import Mystem
m = Mystem()

#print(df['purpose'].unique())
reason1 = m.lemmatize(df['purpose'][0])
#print(reason1)
reason2 = m.lemmatize(df['purpose'][3])
#print(reason2)

def lemmas_purpose(purpose):
    reason = ['жилье','автомобиль','образование','свадьба', 'недвижимость']
    for word in purpose.split(' '):
        lemmas = m.lemmatize(word)
        if lemmas[0] in reason:
            return lemmas[0]
df['lemmas_purpose'] = df['purpose'].apply(lemmas_purpose)
df.loc[df['lemmas_purpose'] == 'жилье','lemmas_purpose'] = 'недвижимость'
print(df['purpose'].unique())
print(df['lemmas_purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
['недвижимость' 'автомобиль' 

### Вывод

Был проанализирован столбец purpose и выявленно 4 категории целей получения кридита. Анализ был произведен с помощью метода unique(). Далее с помощью лемматизации было приведены слова к его словарной форме (лемме). 
Затем была написана функция upgrade_purpose, чтобы разбить цели получения кридита на 4 уникальный категории.

### Категоризация данных

In [103]:
#Категория зависимости количества детей и возврата кредита
def children_debt(row):
    children = row['children']
    debt = row['debt']
    
    if children < 1:
        if debt ==1:
            return "не способен вовремя выплачивать"
    if children > 2:
        if debt ==1:
            return "не способен вовремя выплачивать, многодетная семья"
    if children < 3:
        if debt ==1:
            return "не способен вовремя выплачивать, есть дети"
    return "вовремя выплачивает"

df['children_debt'] = df.apply(children_debt, axis=1)
print(df['children_debt'].value_counts())

#Категория зависимости семейного положения и возврата кредита
def family_status_debt(row):
    family_status = row['family_status']
    debt = row['debt']
    
    if family_status == 'женат / замужем':
        if debt ==1:
            return "не способен вовремя выплачивать, женат / замужем"
    if family_status == 'гражданский брак':
        if debt ==1:
            return "не способен вовремя выплачивать, гражданский брак"
    if family_status == 'вдовец / вдова':
        if debt ==1:
            return "не способен вовремя выплачивать, вдовец / вдова"
    if family_status == 'в разводе':
        if debt ==1:
            return "не способен вовремя выплачивать, в разводе"
    if family_status == 'не женат / не замужем':
        if debt ==1:
            return "не способен вовремя выплачивать, не женат / не замужем"

    return "вовремя выплачивает"

df['family_status_debt'] = df.apply(family_status_debt, axis=1)
print(df['family_status_debt'].value_counts())

#Категория зависимости уровня дохода и возврата кредита
def total_income_debt(row):
    total_income = row['total_income']
    debt = row['debt']
    
    if (total_income < 30000) and (total_income > 0):
        if debt ==0:
            return "вовремя выплачивает, нижний класс"
    if total_income >= 60000:
        if debt ==0:
            return "вовремя выплачивает, верхний класс"
    if (total_income >= 30000) and (total_income < 60000):
        if debt ==0:
            return "вовремя выплачивает, cредний класс"
    return "не способен вовремя выплачивать"

df['total_income_debt'] = df.apply(total_income_debt, axis=1)
print(df['total_income_debt'].value_counts())

#Категория зависимости целей получения кредита и его возврат в срок
def purpose_debt(row):
    purpose = row['lemmas_purpose']
    debt = row['debt']
    
    if purpose == 'недвижимость':
        if debt ==1:
            return "вовремя выплачивает, цель недвижимость"
    if purpose == 'автомобиль':
        if debt ==1:
            return "вовремя выплачивает, цель автомобиль"
    if purpose =='образование':
        if debt ==1:
            return "вовремя выплачивает, цель образование"
    if purpose =='свадьба':
        if debt ==1:
            return "вовремя выплачивает, цель свадьба"
    return "не способен вовремя выплачивать"

df['purpose_debt'] = df.apply(purpose_debt, axis=1)
print(df['purpose_debt'].value_counts())

вовремя выплачивает                                   19713
не способен вовремя выплачивать                        1063
не способен вовремя выплачивать, есть дети              639
не способен вовремя выплачивать, многодетная семья       39
Name: children_debt, dtype: int64
вовремя выплачивает                                       19713
не способен вовремя выплачивать, женат / замужем            931
не способен вовремя выплачивать, гражданский брак           388
не способен вовремя выплачивать, не женат / не замужем      274
не способен вовремя выплачивать, в разводе                   85
не способен вовремя выплачивать, вдовец / вдова              63
Name: family_status_debt, dtype: int64
вовремя выплачивает, верхний класс    17023
не способен вовремя выплачивать        3674
вовремя выплачивает, cредний класс      737
вовремя выплачивает, нижний класс        20
Name: total_income_debt, dtype: int64
не способен вовремя выплачивать           19713
вовремя выплачивает, цель недвижимость   

### Вывод

Данные были разделеные на следующие категории:

    1)Категория зависимости количество детей и возвратом кредита - была создана функция children_debt для фильрации данных и примена функция для каждой строки с помощью метода apply(): он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента.
    
    2)Категория зависимости семейному положению и возвратом кредита - была создана функция family_status_debt для фильрации данных и примена функция для каждой строки с помощью метода apply(): он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента.
    
    3)Категория зависимости уровнем дохода и возвратом кредита - была создана функция total_income_debt для фильрации данных и примена функция для каждой строки с помощью метода apply(): он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента.
    
    4)Категория зависимости по целям получения кредита и его возврат в срок - была создана функция purpose_debt для фильрации данных и примена функция для каждой строки с помощью метода apply(): он берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [104]:
def children_count(row):
    children = row['children']
    
    if children < 1:
        return "нет детей"
    if children > 2:
        return "многодетная семья"
    if children < 3:
        return "есть дети"
df['children_count'] = df.apply(children_count, axis=1)

df_children_debt = df.groupby(['children_count', 'children_debt'])['children'].count()/df.groupby('children_count')['children'].count()
print(df_children_debt)

print('Конверсия многодетной семьи, которые имели задолжность: {:.0%}'.format(df_children_debt[3]))
print('Конверсия семьи с детьми, которые имели задолжность: {:.0%}'.format(df_children_debt[1]))
print('Конверсия семьи без детей, которые имели задолжность: {:.0%}'.format(df_children_debt[5]))


children_count     children_debt                                     
есть дети          вовремя выплачивает                                   0.907485
                   не способен вовремя выплачивать, есть дети            0.092515
многодетная семья  вовремя выплачивает                                   0.914474
                   не способен вовремя выплачивать, многодетная семья    0.085526
нет детей          вовремя выплачивает                                   0.924562
                   не способен вовремя выплачивать                       0.075438
Name: children, dtype: float64
Конверсия многодетной семьи, которые имели задолжность: 9%
Конверсия семьи с детьми, которые имели задолжность: 9%
Конверсия семьи без детей, которые имели задолжность: 8%


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [105]:

df_family_status_debt = df.groupby(['family_status', 'family_status_debt'])['family_status'].count()/df.groupby('family_status')['family_status'].count()
print(df_family_status_debt)

print('Конверсия женатого / замужнего заказчика, который имел задолжность: {:.0%}'.format(df_family_status_debt[7]))
print('Конверсия заказчика в гражданском браке, который имел задолжность: {:.0%}'.format(df_family_status_debt[5]))
print('Конверсия заказчика вдовца/вдовы, который имел задолжность: {:.0%}'.format(df_family_status_debt[3]))
print('Конверсия заказчика в разводе, который имел задолжность: {:.0%}'.format(df_family_status_debt[1]))
print('Конверсия не женатого / не замужнего заказчика, который имел задолжность: {:.0%}'.format(df_family_status_debt[9]))

family_status          family_status_debt                                    
в разводе              вовремя выплачивает                                       0.928870
                       не способен вовремя выплачивать, в разводе                0.071130
вдовец / вдова         вовремя выплачивает                                       0.934307
                       не способен вовремя выплачивать, вдовец / вдова           0.065693
гражданский брак       вовремя выплачивает                                       0.906529
                       не способен вовремя выплачивать, гражданский брак         0.093471
женат / замужем        вовремя выплачивает                                       0.924548
                       не способен вовремя выплачивать, женат / замужем          0.075452
не женат / не замужем  вовремя выплачивает                                       0.902491
                       не способен вовремя выплачивать, не женат / не замужем    0.097509
Name: family_status, d

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [106]:
def total_income_count(row):
    total_income = row['total_income']
    
    if (total_income < 30000) and (total_income > 0):
        return "нижний класс"
    if total_income >= 60000:
        return "верхний класс"
    if (total_income >= 30000) and (total_income < 60000):
        return "cредний класс"

df['total_income_count'] = df.apply(total_income_count, axis=1)

df_total_income_debt = df.groupby(['total_income_count', 'total_income_debt'])['total_income'].count()/df.groupby('total_income_count')['total_income'].count()
print(df_total_income_debt)

print('Конверсия уровня дохода нижнего класса, которые не имеют задолжность: {:.0%}'
      .format(df_total_income_debt[4]))
print('Конверсия уровня дохода верхнего класса, которые не имеют задолжность: {:.0%}'
      .format(df_total_income_debt[2]))
print('Конверсия уровня дохода среднего класса, которые не имеют задолжность: {:.0%}'
      .format(df_total_income_debt[0]))


total_income_count  total_income_debt                 
cредний класс       вовремя выплачивает, cредний класс    0.940051
                    не способен вовремя выплачивать       0.059949
верхний класс       вовремя выплачивает, верхний класс    0.917929
                    не способен вовремя выплачивать       0.082071
нижний класс        вовремя выплачивает, нижний класс     0.909091
                    не способен вовремя выплачивать       0.090909
Name: total_income, dtype: float64
Конверсия уровня дохода нижнего класса, которые не имеют задолжность: 91%
Конверсия уровня дохода верхнего класса, которые не имеют задолжность: 92%
Конверсия уровня дохода среднего класса, которые не имеют задолжность: 94%


- Как разные цели кредита влияют на его возврат в срок?

In [107]:
df_purpose_debt = df.groupby(['lemmas_purpose', 'purpose_debt'])['lemmas_purpose'].count()/df.groupby('lemmas_purpose')['lemmas_purpose'].count()
print(df_purpose_debt)

print('Конверсия цели кредита недвижимость, которые имеют задолжность: {:.0%}'
      .format(df_purpose_debt[3]))
print('Конверсия цели кредита автомобиль, которые имеют задолжность: {:.0%}'
      .format(df_purpose_debt[1]))
print('Конверсия цели кредита образование, которые имеют задолжность: {:.0%}'
      .format(df_purpose_debt[5]))
print('Конверсия цели кредита свадьба, которые имеют задолжность: {:.0%}'
      .format(df_purpose_debt[7]))


lemmas_purpose  purpose_debt                          
автомобиль      вовремя выплачивает, цель автомобиль      0.093590
                не способен вовремя выплачивать           0.906410
недвижимость    вовремя выплачивает, цель недвижимость    0.072334
                не способен вовремя выплачивать           0.927666
образование     вовремя выплачивает, цель образование     0.092200
                не способен вовремя выплачивать           0.907800
свадьба         вовремя выплачивает, цель свадьба         0.080034
                не способен вовремя выплачивать           0.919966
Name: lemmas_purpose, dtype: float64
Конверсия цели кредита недвижимость, которые имеют задолжность: 93%
Конверсия цели кредита автомобиль, которые имеют задолжность: 91%
Конверсия цели кредита образование, которые имеют задолжность: 91%
Конверсия цели кредита свадьба, которые имеют задолжность: 92%


### Вывод

Ответы на вопросы:

1)Зависимости количество детей и возвратом кредита - Анализ показал, что если у заказчика нет детей, то шанс, что задолжности не будет, выше, чем у заказчика с детьми. Так же анализ показал, что у заказчика, если есть денти, то их количество не влияет на задолжность кредита.

2)Зависимости семейному положению и возвратом кредита - Анализ показал, что "вдовец/вдова" и "в разводе" более ответсвенее и меньше всех имею задолдность по возврату в срок. А вот не женатый / не замужний закзчик больше всех имеет задолжность по возврату кредита 

3)Зависимость между уровнем дохода и возвратом кредита в срок - Анализ показал что "средний класс" более ответсвенный к распределению дохода и поэтому имеет больше процентов заказчиков, которые не имеют задолжность по кредиту 

4)Зависимость между разных цели кредита и его возвратом в срок - Анализ показал что заказчик, который берет кредит на образование или на автомобиль имеет меньше всего процентов по задолжности кредита в банке. А заказчик который берет кредит на "недвижимость/жилье" имеет большего всего процент задолжности кредита в банке.

### Шаг 4. Общий вывод

Исходя из проделанной работы можно сделать следующие выводы :

1) Поля для записей стажа работы, количества детей не могут быть отрицательными;

2) Единственная категория, имеющая первую букву верхнего регистра - Не женат/не замужем;

3) Важен определенный тип данных для каждой категории, например типа данных float не подходит для стажа работы (правильно - integer);

4) Лемматизация обязательна для разбивки множества слов на различные значительные категории;

5) Для категоризации очень удобен метод apply - он применяет свою функцию ко всем значениям датафрейма;

Ответы на вопросы : 

1) Проанализировав зависимость наличия детей и возвратом кредита в срок можно сделать вывод, что конверсия семей с детьми не значительно отличается от конверсии бездетной семьи по возвратам кредита в срок;

2) Оценив зависимость семейного положения и возврата кредита в срок можно сказать, что более ответственны и имеющие минимальную задолженность категория лиц, являющихся вдвцами/вдвовами, либо же те, кто в разводе. Однако, наибольшую задолженность по кредиту имеет категория "не женат\не замужем";

3) Пересмотрев зависимость между уровнем дохода и возврата кредита в срок, можно увидеть, что лица, относящиеся к категории "среднего класса" более ответственны к распределению дохода и имеют высокий процент заказчиков, не имеющих задолженностей по кредиту;

4) Анализ показал, что заказчик, берущий кредит на более дорогостоящие вещи(недвижимость) имеет наибольший процент по задолженности кредита в банке, нежели заказчик, покупающий автомобиль или на образование.    

Данный анализ рекомендуется использовать отделениям банка по выдаче кредитов. Мы видим, что заказчики с детьми как правило 9% имеют задолжность в банке против 8% задолжников без детей. Это говорит о трудном финансовом положении заказчика. Также, из анализа следует, что заказчик имеющий доход ниже среднего класса имеют задолжности 9%. Это потенциально опасно для отделений банка, выдающим заказчикам кредит.

Рекомендуется выдавать кредит лицам:

    1)не имеющих детей;
    2)состоявших в браке;
    3)среднего класса и выше;
    4)с целью покупки образования или на автомобиль


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.